# CO₂ Emissions per Square Kilometer in European Countries
In this notebook we will be plotting a few visualizations for CO2 emissions per square kilometer in European Countries throughout the years from 1970-2023.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

### Data loading

To begin the project, we explored available databases related to pollution and CO2 emissions. After gathering the data from online sources, we organized it into appropriate folders and integrated it into our code.

In [3]:
data_root = './data/'

In [4]:
# Reading the data
df_co2 = pd.read_csv(f'{data_root}co2_emmisions_complicated.csv')

As the dataset included information on countries worldwide, we decided to narrow our focus to a more manageable scope by analyzing data specifically related to European countries.

In [23]:
# Filtering the data only for european countries
df_co2_europe = df_co2[df_co2['Region'].str.contains('Europe', case=False, na=False)]

# Filter Russia
df_russia = df_co2[df_co2['Name'].str.contains('Russian Federation', case=False, na=False)].copy()

# Rename 'Russian Federation' to 'Russia'
df_russia['Name'] = df_russia['Name'].replace('Russian Federation', 'Russia')

# Concatenate and remove duplicates if needed
df_co2_europe = pd.concat([df_co2_europe, df_russia]).drop_duplicates()

In [24]:
countries = df_co2_europe["Name"].unique().tolist()
print(countries)

['Albania', 'Austria', 'Belgium', 'Bulgaria', 'Bosnia and Herzegovina', 'Switzerland', 'Cyprus', 'Czech Republic', 'Germany', 'Denmark', 'Spain', 'Estonia', 'Finland', 'France', 'United Kingdom', 'Greece', 'Croatia', 'Hungary', 'Ireland', 'Iceland', 'Italy', 'Lithuania', 'Luxembourg', 'Latvia', 'North Macedonia', 'Malta', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Serbia and Montenegro', 'Slovakia', 'Slovenia', 'Sweden', 'Russia']


In [25]:
countries_area = {
    'Albania': 28748,
    'Austria': 83871,
    'Belgium': 30528,
    'Bulgaria': 110879,
    'Bosnia and Herzegovina': 51197,
    'Switzerland': 41285,
    'Cyprus': 9251,
    'Czech Republic': 78866,
    'Germany': 357022,
    'Denmark': 43094,
    'Spain' : 505978,
    'Estonia': 45340,
    'Finland': 338470, 
    'France': 549087,
    'United Kingdom': 243610,
    'Greece': 131960,
    'Croatia': 88070,
    'Hungary': 93030,
    'Ireland': 70280,
    'Iceland': 103000,
    'Italy': 302070,
    'Lithuania': 65286,
    'Luxembourg': 2590,
    'Latvia': 64590,
    'North Macedonia': 25710,
    'Malta': 320,
    'Netherlands': 41540,
    'Norway': 624500,
    'Poland': 312720,
    'Portugal': 92230, 
    'Romania': 238400,
    'Serbia and Montenegro': 98800,
    'Slovakia': 49030,
    'Slovenia': 20480,
    'Sweden': 528860,
    'Russia': 17098250 
}